# ライブラリの読み込み

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adadelta, Adam

import numpy as np
import random
import sys

# 準備

In [ ]:
from tqdm import tqdm
import music21 as m21
import os

BasePath = "../midi/元データ"
MelodyPath = BasePath + "/MelodyTrackList.txt" #主旋律
ChordPath = BasePath + "/ChordTrackList.txt" #コード
CMelodyPath = BasePath + "/CMelodyTrackList.txt" #対旋律

music_keys = ('C')

TrackList = {}
with open(ChordPath) as f:
    for line in f.readlines():
        s = line.strip().split("=")
        TrackList[s[0]] = {"chord":s[1].split(",")}
    
with open(CMelodyPath) as f:
    for line in f.readlines():
        s = line.strip().split("=")
        TrackList[s[0]]["cmelody"] = s[1].split(",")

#今回はChordTrackListに対してMelodyTrackListの要素数が少ないので、簡略的にこうしている
#同じだった場合は、CMelodyと同様に書く
for song in TrackList.keys(): 
    TrackList[song]["melody"] = ["1"]

print(TrackList)

# 音符・休符の文字列への変換

In [ ]:
#トラックの結合
def concatenateTracks(path, trackNums):
    track = m21.converter.parse(path)
    p = m21.stream.Part(id="part")
    for i in trackNums:
        piece = track.parts[int(i)-1]
        s = piece.measures(0,None)
        p.insert(0,s)
        
    return p

In [ ]:
#曲が始まってからの時間でそのときの主旋律の音名を調べる辞書を作成
TimingToPitch = {}
for path in TrackList.keys():
    track = m21.converter.parse(path)
    piece = track.parts[0] #今回は主旋律=1トラック目の曲しかない
    k = piece.analyze('key')
    trans = 'C'
    i = m21.interval.Interval(k.tonic, m21.pitch.Pitch(trans))
    trans_piece = piece.transpose(i)

    TimingToPitch[path] = np.zeros(int(track.quarterLength))
    for n in trans_piece.flat.notes:
        TimingToPitch[path][int(n.offset)] = n.pitch.midi % 12


In [ ]:
def TrackToStrList(path, trackNums):
    track = m21.converter.parse(path)
    piece = concatenateTracks(path, trackNums) #コードなので結合
    
    for trans_key in music_keys:
        k = piece.analyze('key')
        trans = trans_key
        
        i = m21.interval.Interval(k.tonic, m21.pitch.Pitch(trans))
        trans_piece = piece.transpose(i)
        preOffset = 0
        TrackStr = []
        for n in trans_piece.flat.notes:
            notes = []
            if isinstance(n, m21.note.Note):
                notes = [n]
            elif isinstance(n, m21.chord.Chord):
                notes = [x for x in n]
            
            restLen = n.offset - preOffset
            for note in notes:
                TrackStr.append(str(note.pitch.midi % 12 -TimingToPitch[path][int(note.offset)]) + '_' + str(n.duration.quarterLength) + '_' + str(restLen) + ' ')
                restLen = 0
            preOffset = n.offset
    return TrackStr

In [ ]:
# コードトラックをつなげる
chars = []
for track in TrackList.keys():
    chars.extend(TrackToStrList(track, TrackList[track]['chord']))

# 辞書の作成

In [ ]:
count = 0
char_indices = {} #辞書
indices_char = {} #逆引き辞書

for word in chars:
    if not word in char_indices:
        char_indices[word] = count #key=word, value=count
        count += 1
        print(count, word)

#逆引き辞書の作成
indices_char = dict([(value, key) for (key, value) in char_indices.items()])

# 文章の作成

In [ ]:
maxlen = 10
step = 1
sentences = []
next_chars = []
text = chars

for i in range(0, len(text)-maxlen, step): #初項0, 末項len..., 公差step
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print("nb sequences:", len(sentences))

# One-Hotベクトル化

In [ ]:
print("Vectorization...")
input_data = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
output_data = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        input_data[i, t, char_indices[char]] = 1 #sentence番号, sentence内のindex, 辞書での項目番号
    output_data[i, char_indices[next_chars[i]]] = 1 #正解データ

# モデルの作成・学習

In [ ]:
epochs = 200
print("Build model...")
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
stack = model.fit(input_data,output_data,batch_size=64,epochs=epochs,validation_split=0.2)
model.save("acway1.h5")

# 学習曲線の描画

In [ ]:
import matplotlib.pyplot as plt

x = range(epochs)

plt.plot(x, stack.history['accuracy'], label="accuracy")
#plt.plot(x, stack.history['val_accuracy'], label="val_accuracy", color="orange")
plt.title("accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.plot(x, stack.history['loss'], label="loss")
#plt.plot(x, stack.history['val_loss'], label="val_loss", color="orange")
plt.title("loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

print(stack.history.keys())

# 伴奏の生成

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) #softmax関数の計算
    probas = np.random.multinomial(1, preds, 1) 
    #http://www.gentosha-academy.com/serial/okamoto-4/ にmultinomialの説明あり
    #今回は1からkがpreds=[p1,p2,...,pk]という度数分布に従っているとき、
    #1回(var1)試行を行ったときの度数分布が1サンプル(var3)得られる
    return np.argmax(probas) #1になっている要素番号を返す

In [ ]:
def make_melody(length=200):
    #適当にスタートの文を選ぶ
    start_index = random.randint(0, len(text)-maxlen-1)
    
    for diversity in [0.5]:
        generated = ''
        sentence = text[start_index:start_index+maxlen] #ここでユーザの伴奏を与えると続きを生成
                   
        generated += ''.join(sentence) #sentence(list)の各要素を結合してgeneratedに追加
        print(sentence)
        
        for i in range(length):
            input_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                input_pred[0, t, char_indices[char]] = 1
            
            preds = model.predict(input_pred, verbose=0)[0] #verbose:詳細 0で詳細情報を表示しない
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            generated += next_char
            sentence = sentence[1:]
            sentence.append(next_char)
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    return generated

#model = load_model('acway1.h5')
melo_sentence = make_melody(100)
print(melo_sentence)

# 出力

In [ ]:
#文字列をmidiに変換
meas = m21.stream.Stream()
meas.append(m21.meter.TimeSignature('4/4'))
melo = melo_sentence.split()
offset = 0
for m in melo:
    pitch, length, _offset = m.split('_')
    
    tmp = length.split('/')
    if len(tmp) == 2:
        length = float(tmp[0])/float(tmp[1])
    else:
        length = float(tmp[0])
    
    offset += float(_offset)
    n = m21.note.Note(TimingToPitch['../midi/元データ\\aogeba.mid'][int(offset)]+int(float(pitch))+60, quarterLength=length)
    #ユーザの与えた主旋律が仰げば尊しだと仮定
    meas.insert(offset,n)

meas.makeMeasures()
meas.show("midi")
meas.write(fmt="midi", fp="./way2.mid")